# Model Training

This notebook demonstrates training the LSTM model for gesture-based text prediction.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import tensorflow as tf
from src.data.generator import DataGenerator
from src.models.lstm_model import create_model
from src.training.trainer import Trainer
from src.config import Config

## Configuration

In [ ]:
# Load configuration
config = Config()
print(f'Model configuration loaded')
print(f'Sequence length: {config.sequence_length}')
print(f'Batch size: {config.batch_size}')

## Generate Training Data

In [ ]:
# Initialize data generator
generator = DataGenerator(config)
train_data, val_data = generator.generate_dataset()
print(f'Training samples: {len(train_data)}')
print(f'Validation samples: {len(val_data)}')

## Build and Train Model

In [ ]:
# Create model
model = create_model(config)
model.summary()

In [ ]:
# Train model
trainer = Trainer(model, config)
history = trainer.train(train_data, val_data)

# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## Save Model

In [ ]:
# Save trained model
model.save('../models/checkpoints/gesture_model.h5')
print('Model saved successfully')